In [ ]:
## hack for missing ESMFMKFILE
import os
os.environ['ESMFMKFILE']='/home/fiaz/anaconda3/envs/aos112/lib/esmf.mk'

import xarray as xr
from glob import glob
import netCDF4
import matplotlib.pyplot as plt
import numpy as np
import xesmf as xe
import datetime as dt
from dateutil.relativedelta import relativedelta
from itertools import repeat
%matplotlib inline

In [ ]:
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)
import thermodynamic_functions

In [ ]:
from dask.distributed import Client,LocalCluster
# client = Client(n_workers=10,)  # start distributed scheduler locally.  Launch dashboard
cluster = LocalCluster(n_workers=10, threads_per_worker=2)
client = Client(cluster, asynchronous=True)  # start distributed scheduler locally.  Launch dashboard
client

In [ ]:
import imerg_era5_preprocessing
import importlib
importlib.reload(imerg_era5_preprocessing)
from imerg_era5_preprocessing import ProcessEra5,ProcessImerg

In [ ]:
fils_era5=glob('/neelin2020/ERA-5_Tq_ps/hourly/*grib')
fils_era5_idx=glob('/neelin2020/ERA-5_Tq_ps/hourly/*idx')
fils_era5.sort()
fils_era5=[fil for fil in fils_era5 if fil not in fils_era5_idx]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DIR_OUT='/neelin2020/ML_input/imerg-era5' ### full precip.
DIR_IMERG='/neelin2020/IMERG_V06'

Define functions

In [ ]:
SAVE_REGION='OCN' ##LND
FILTER_SURF_PRESS=800 ## filter points with surface pressure < 800 hPa

In [ ]:
def process_era5_imerge_regridded_files(fils_era5):
    for filx in fils_era5:
        
        print(filx)
        ## Process imerg data ###
        obj_imerg=ProcessImerg(filx,SAVE_REGION,DIR_OUT,DIR_IMERG,
                                FILTER_SURF_PRESS)        
        obj_imerg.check_imerge()
        if obj_imerg.imerg_processing:
            print('Processing IMERG')
            obj_imerg.save_imerg()
        else:
            print('IMERG files exist')
        
        ## extract ERA5 datasets ###
        print('Working on ERA5')
        obj_era5=ProcessEra5(filx,SAVE_REGION,DIR_OUT,
                              FILTER_SURF_PRESS) 
        obj_era5.check_era5_vars()
        
        if obj_era5.era5_processing:
            print('ERA5 processing required')
            obj_era5.extract_era5()
            print('Computing ERA5 thermo variables')
            obj_era5.compute_thermo()
            print('Computing layer-averaged variables')
            obj_era5.get_layer_averaged_thermo()
            print('Saving ERA5 variables')
            obj_era5.save_era5_thermo()
        else:
            print('ERA5 files exist')
        
        
    return 

In [ ]:
obj=process_era5_imerge_regridded_files(fils_era5[0:20])